In [7]:
using Random, PyPlot, Statistics, JLD, StatsBase, Distributions
using LinearAlgebra

const T = Float64 # just getting the picture
const p = T.((180//100,35//100))

include("../simulations/cool-intervals.jl")
L = load("../simulations/Conjecture2.jld")
T==Float64 && (endmean = convert(T,mid(2mean(L["tobsv"] .* L["wrespv"]) / mean(L["wrespv"]))))

# perfect obs stuff
    correlf = 2Interval{Float64}.(L["trespv"] .- L["tobsv"] .* L["wrespv"])
    correlf_m = vec(mean(correlf,dims=1))./mean(Interval{Float64}.((L["wrespv"])))
    correlf_s = vec(mapslices(sem,correlf,dims=1))./mean(Interval{Float64}.((L["wrespv"])))
    correlf_ci = quantile(TDist(99),1-0.01/100) * correlf_s;

┌ Warning: JLD incorrectly extends FileIO functions (see FileIO documentation)
└ @ FileIO /Users/caro/.julia/packages/FileIO/JA3Vl/src/loadsave.jl:215


In [8]:
function fuzzy_sampler(noise_std,len=200_000,obs=0)
    # start with the backward-invariant unstable manifold between w_0 and w_1
    beta = (p[1]-sqrt(p[1]^2+4p[2]))/2; alpha = -beta/(1+beta)
    x_ref_end = -alpha/beta; x_ref_start = 1-p[1]*x_ref_end
    Q = [1 -beta; beta 1]/sqrt(1+beta*beta)
    Is = Q[2,:]*alpha .+ sqrt(1+beta*beta)*T.([1,0])*[x_ref_start x_ref_end]

    # spinup
    for i = 1:10000
        Is, Q, c, cw = cool_intervals(Is,Q,p)
    end
    
    attractor_endpts = Array{Float64}(undef,2,2,4000)
    ch = Array{Float64}(undef,len)
    cwh = Array{Float64}(undef,len)
    attractor_endpts = Array{Float64}(undef,2,2,4000)
    samplepth = fill(NaN,2,len)
    sampleptw = fill(NaN,len)
    for i = 1:len
        Is, Q, c, cw = cool_intervals(Is,Q,p)
        ptx = obs+randn()*noise_std
        QIs = Q*Is
        i<=4000 && (attractor_endpts[:,:,i] = Float64.(mid.(QIs)))
        ch[i] = c
        cwh[i] = cw
        if maximum(QIs[1,:]) >= ptx >= minimum(QIs[1,:])
            samplepth[1,i] = ptx
            QIs_shft = QIs .- [ptx,0];
            samplepth[2,i] = (QIs_shft[2,1]*QIs_shft[1,2]-QIs_shft[1,1]*QIs_shft[2,2])/(QIs_shft[1,2]-QIs_shft[1,1])
            sampleptw[i] = 1/abs(Is[1,1]-Is[1,2]) # this is wrong, but what can you do
        end
    end
    return samplepth, sampleptw, attractor_endpts, ch, cwh
end

fuzzy_sampler (generic function with 3 methods)

In [9]:
# sampK = 50
# samplesizes = [10.0 .^(-(1:2:7));0]
# runs = 10
# sampleconv = Array{Float64}(undef,runs,sampK,length(samplesizes))
# for j in eachindex(samplesizes)
#     for r = 1:runs
#         samplepth, sampleptw, attractor_endpts, ch, cwh = fuzzy_sampler(samplesizes[j],div(4*10^7,runs));
#         samplepth = samplepth[:,.~isnan.(samplepth[1,:])]
#         sampleptw = sampleptw[.~isnan.(sampleptw)]
#         for k = 1:sampK
#             sampleconv[r,k,j] = 2mean(samplepth[1,:].*sampleptw)/mean(sampleptw)
#             if k!=sampK 
#                 for i = 1:size(samplepth,2)
#                     samplepth[:,i] .= smap!(samplepth[:,i],p)
#                 end
#             end
#         end
#     end
# end
# sampleconv .-= endmean;
# save("../simulations/sampleconv.jld","sampleconv",sampleconv,"samplesizes",samplesizes,"sampK",sampK)
@load "../simulations/sampleconv.jld"

3-element Vector{Symbol}:
 :sampK
 :sampleconv
 :samplesizes

In [14]:
figure(figsize=(5.76*0.8,3))
n=3
    n==3 && errorbar(0:L["K"]-1,abs.(mid.(abs.(correlf_m))),sup.(correlf_ci),c="k",label="Perfect obs.")
    for j in (n==1 ? [1] : reverse(eachindex(samplesizes))[2:end])
        errorbar(0:sampK-1,abs.(vec(mean(sampleconv[:,:,j],dims=1))-mid.(0correlf_m[1:50])),
            quantile(TDist(99),1-0.01/100) * vec(mapslices(sem,sampleconv[:,:,j],dims=1)),
            label="\$ \\sigma = 10^{$(round(Int,log10(samplesizes[j])))}\$",c="C$(j-1)")
    end
    semilogy()
    legend()
    xlabel("\$n\$"); ylabel("\$|\\mu^+_0(A\\circ T^n) - \\rho(A)|\$")
    # fill_between(0:L["K"]-1,inf.(abs.(correlf_m)),sup.(abs.(correlf_m)))
#     title("Predicted expectations of \$A(x,y) = 2x\$ with \$\\sigma\$-noise")
    xlim(0,sampK-1);ylim(ymin=1.7e-3)
    tight_layout()
    savefig("../figures/Bayesian.pdf")


In [36]:
figure(figsize=(5.76*0.7,4))
for (n,sigma) = enumerate((0.1,0.03))
    subplot(210+n)
    obs = sigma==0 ? 0 : sigma*randn(MersenneTwister(n+5))

    samplepth, sampleptw, attractor_endpts, ch, cwh = fuzzy_sampler(sigma,4000,obs)
    lines = [Float64.(mid.(attractor_endpts[:,:,i]')) for i = 1:500]
    lwl = [1 ./norm(l[1,:] - l[2,:]) for l in lines]; lwl /= 3mean(lwl); lwl = min.(lwl,1)
    attractor = matplotlib.collections.LineCollection(lines,
            linewidth=1.0,colors=[zeros(length(lines),3) lwl],zorder=1)
    gca().add_collection(attractor)
    plot([0],[0],c=[0,0,0,0]); yl = ylim() # plotting something quite dummy just to get the attractor's ylim
    plot([obs,obs],[-10,10],"C4",label="\$\\mathcal{S}\$",zorder=1); ylim(yl)
    # gca().set_aspect(1/p[2])
    xlabel("\$x\$"); ylabel("\$y\$")
    text(obs+0.01,0.37,"\$H_{\\mathrm{obs}} = $(round(obs,sigdigits=2))\$")
    # title("Lozi attractor")
    if sigma == 0
        scatter(zeros(1000),mid.(ch[1:1000]),c="C1",s=1,zorder=3)
    else
        scatter(samplepth[1,:],samplepth[2,:],s=1,alpha=3sigma,ec="none",c="C1",zorder=3)
    end
    text(-1.24,-0.38, "\$\\sigma = $(sigma)\$",size=12)
end
tight_layout()
savefig("../figures/BayesianAttractor.pdf")
